In [1]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output

import wandb
import pprint


from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor, IDSWDataSetLoader
from architectures import vgg16net, smallnet1, smallnet2, smallnet3
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, hp_sweep, train_model, train_log, log_test_score, set_lossfn, pipeline

from architectures import build_net, smallnet3


In [2]:
"""
'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 1e-5,
                'max': 1e-2
            },
"""

"\n'learning_rate': {\n                # a flat distribution between 0 and 0.1\n                'distribution': 'log_uniform_values',\n                'min': 1e-5,\n                'max': 1e-2\n            },\n"

In [24]:
"""config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        'dropout':{
            'values': [0.5]
        },
        'weight_decay':{
            'values': [3.00E-05]
        },
        'epochs':{
            'value': 120
        },
        'lin_layer_size': {
            'values': [100] #, 150, 50
        },
        'first_lin_lay':{
            'values':[11264]
        },
        'optimizer': {
            'values': ['adam']
        },
            'learning_rate': {
                'values': [9.76E-05]
            },
        'loss_fn': {
            'values': ['MSE', 'CrossEntropy'] #
        },
        'data_set':{
            'values':['Augmented']
        },
            'scheduler': {
            'values': [0.2]
        },
        'ks': {
            'values': [(3,5)]
        },
        'channels':{
            'values': [3]
        },
        'num_classes': {
            'values': [11]
        },
        'model_name' : {'values': ['vgg16net']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/']
        },
        }
    }
"""

config = dict(
    epochs= 120, 
    learning_rate = 9.76E-05, #9e-5,#3.0779e-5,
    dataset= 'IDSW',
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    dropout = 0.3,
    first_lin_lay = 248832, #59904, #11264,#248832, #3981312, #248832,
    lin_layer_size= 100,
    ks = (3,5),
    in_chan = 3,
    image_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/',
    model_name = 'vgg16net',
    channels = 3,
    num_classes = 11,
    loss_fn = 'CrossEntropy',
    scheduler = 0.5
)
"""
title = f'BATCHTEST_IDSW_7c3l_150_300124'
save_dict = {'Run' : title,
            'Current_Epoch': 0,
            'save_location' : r'pickles/'}"""
#pickles


col_dict = {
    'colour': 'colour',
    'size': [452,144],
    'padding': 5
}

title = f"IDSWAug_7c3l_E120_{col_dict['size']}_310124"
save_dict = {'Run' : title,
            'Current_Epoch': 0,
            'save_location' : r'pickles/'}

# modelSize_7conv3lin_IDSW
#sweep_id = wandb.sweep(config, project=title+f"_{col_dict['colour']}_{col_dict['size']}_{col_dict['padding']}")

In [4]:
"""import pprint 
pp = pprint.PrettyPrinter()
pp.pprint(config)
print(type(config))
pp.pprint(col_dict)
print(type(col_dict))

c = col_dict['colour']
print(c)"""

"import pprint \npp = pprint.PrettyPrinter()\npp.pprint(config)\nprint(type(config))\npp.pprint(col_dict)\nprint(type(col_dict))\n\nc = col_dict['colour']\nprint(c)"

In [5]:
import os
print(os.getcwd())

/its/home/nn268/antvis/antvis/optics


In [6]:
#pprint.pprint(config)

In [ ]:
#  image_file_path= r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'

from fns4wandb import train

device = "cuda:1" if torch.cuda.is_available() else "cpu"

model = train(device,col_dict, save_dict, config)

"""def tr(config=None, save_dict=save_dict):
    with wandb.init(config=config):
        seed = random.randint(0, 50)
        config = wandb.config
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model, save_dict, x_test, y_test = hp_sweep(config, col_dict, save_dict, device, seed)
        test_predict_list, test_labels, test_accuracy = test_loop(model, x_test, y_test, loss_fn, device, col_dict, title, config.num_classes)
        save_dict['test_predict_list'] = [c.to('cpu') for c in test_predict_list]
        save_dict['test_labels'] = [c.to('cpu') for c in test_labels]
        save_dict['test_accuracy'] = test_accuracy
        save_dict['seed'] = seed
        title = save_dict['Run']
        with open(f"/its/home/nn268/antvis/antvis/optics/pickles/{title}.pkl", 'wb+') as f:
            pickle.dump(save_dict, f)

wandb.agent(sweep_id, tr, count=25)"""

#model= pipeline(config, col_dict,save_dict, title, device)

  0%| | 0/1/its/home/nn268/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
 33%|▎| 40/

In [8]:
"""def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model = hp_sweep(config, col_dict, save_dict, device)

wandb.agent(sweep_id, tr, count=2)"""

'def tr(config=None):\n    with wandb.init(config=config):\n        config = wandb.config\n        device = "cuda:0" if torch.cuda.is_available() else "cpu"\n        model = hp_sweep(config, col_dict, save_dict, device)\n\nwandb.agent(sweep_id, tr, count=2)'